In [ ]:
# Project 1: Image classifier using Transfer Learning (MobileNetV2) on CIFAR-10
# Paste and run this cell after confirming TensorFlow is available.

# 1) Imports and basic checks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# Print TF version to confirm environment
print("TensorFlow version:", tf.__version__)

# 2) Load CIFAR-10 dataset
# CIFAR-10: 60k images, 32x32 color, 10 classes. Great for quick experiments.
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 3) Preprocess - convert to float32 and scale to [0,1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# y_train and y_test are shape (n,1) - that's okay for sparse_categorical_crossentropy
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# 4) Data augmentation pipeline (real-time augmentation during training)
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),        # flip left-right
        layers.RandomRotation(0.06),            # small rotations
        layers.RandomZoom(0.05),                # small zooms
    ],
    name="data_augmentation",
)

# 5) Build model using MobileNetV2 as a frozen feature extractor
# - We will upscale CIFAR images (32x32) to the input size expected by MobileNetV2.
# - include_top=False: we remove the classifier head and add our own.
base_model = keras.applications.MobileNetV2(
    input_shape=(96,96,3),   # MobileNetV2 expects at least 96x96 for weights used here
    include_top=False,
    weights='imagenet',      # use pretrained ImageNet weights
    pooling='avg'            # global average pooling at end
)
base_model.trainable = False   # freeze the base model initially

# 6) Create final model: input -> upsample -> augmentation -> preprocess -> base -> classifier head
inputs = keras.Input(shape=(32,32,3))          # CIFAR-10 image shape
x = layers.UpSampling2D(size=(3,3))(inputs)    # 32x32 -> 96x96 (approx)
x = data_augmentation(x)                       # apply augmentation only during training
x = keras.applications.mobilenet_v2.preprocess_input(x)  # model-specific preprocessing
x = base_model(x, training=False)              # pass through the frozen base model
x = layers.Dropout(0.3)(x)                     # dropout for regularization
x = layers.Dense(256, activation="relu")(x)    # small dense layer
x = layers.Dropout(0.3)(x)                     # another dropout
outputs = layers.Dense(10, activation="softmax")(x)  # final softmax for 10 classes
model = keras.Model(inputs, outputs)

# 7) Compile the model with optimizer, loss, metrics
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",  # labels are integer class indices
    metrics=["accuracy"],
)

# 8) Show model summary (architecture)
model.summary()

# 9) Train the model (quick run). For better results increase epochs.
history = model.fit(
    x_train, y_train,
    epochs=12,              # start small; increase if you have time/GPU
    batch_size=64,
    validation_split=0.15,  # keep 15% of training data for validation
)

# 10) Optional: fine-tune last layers of the base model for a bit more accuracy
# Unfreeze some of base_model's layers and train with a lower LR
base_model.trainable = True
# Freeze all layers except last N layers to avoid destroying pretrained features
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with a much lower learning rate for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

# Fine-tune for a few more epochs
fine_history = model.fit(
    x_train, y_train,
    epochs=6,
    batch_size=64,
    validation_split=0.15,
)

# 11) Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

# 12) Save model locally (Colab VM). To keep it permanently, mount Google Drive and save there.
model.save("cifar10_mobilenetv2_transfer.h5")
print("Model saved as cifar10_mobilenetv2_transfer.h5")

# 13) Plot training curves for accuracy and loss (combine histories)
# Combine history objects safely (they may have different lengths)
train_acc = history.history.get('accuracy', []) + fine_history.history.get('accuracy', [])
val_acc   = history.history.get('val_accuracy', []) + fine_history.history.get('val_accuracy', [])
plt.figure(figsize=(8,4))
plt.plot(train_acc, label='train_accuracy')
plt.plot(val_acc, label='val_accuracy')
plt.title("Training / Validation Accuracy")
plt.legend()
plt.show()

# 14) Display a few test predictions (visual check)
import random
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

# Predict probabilities for the test set
preds = model.predict(x_test[:25])  # predict first 25 test images
pred_labels = np.argmax(preds, axis=1)

# Plot a 5x5 grid
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[i])
    plt.xlabel(f"pred: {class_names[pred_labels[i]]}\ntrue: {class_names[int(y_test[i])]}")
plt.show()


TensorFlow version: 2.19.0
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
x_train shape: (50000, 32, 32, 3) y_train shape: (50000, 1)
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,588,490 (9.87 MB)

 Trainable params: 330,506 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/12
665/665 ━━━━━━━━━━━━━━━━━━━━ 374s 554ms/step - accuracy: 0.1013 - loss: 2.3929 - val_accuracy: 0.0953 - val_loss: 2.3029
Epoch 2/12
665/665 ━━━━━━━━━━━━━━━━━━━━ 423s 616ms/step - accuracy: 0.0977 - loss: 2.3032 - val_accuracy: 0.0984 - val_loss: 2.3027
Epoch 3/12
665/665 ━━━━━━━━━━━━━━━━━━━━ 417s 627ms/step - accuracy: 0.0998 - loss: 2.3028 - val_accuracy: 0.0953 - val_loss: 2.3027
Epoch 4/12
185/665 ━━━━━━━━━━━━━━━━━━━━ 4:01 502ms/step - accuracy: 0.0994 - loss: 2.3027